In [1]:
import pandas as pd
import numpy as np
np.random.seed(2023 - 6 - 23)
import matplotlib.pyplot as plt
from scipy.stats import t
import os

In [5]:
os.listdir("./../data")

['Cartwheeldata.csv',
 'CommonCLZvalues.png',
 'Data Description_Metro.pdf',
 'EV_Population_Data.csv',
 'MetroPT3(AirCompressor).csv',
 'nap_no_nap.csv',
 'nhanes_2015_2016.csv',
 'USDA_plants_database.csv']

## ___Estimating a Confidence Interval for Population Mean, Using a Sample From The Dataset.___
----------------------

In [2]:
metro = pd.read_csv("./../data/MetroPT3(AirCompressor).csv")

In [9]:
# Let's consider this dataset as our population.
# We are interested in figuring out the mean oil temperature.
# The true population parameter (mean oil temp) is

metro.Oil_temperature.mean()

60.91453961453962

In [12]:
# However, in real world scenarios, we cannot measure this true population parameter of interest.
# The size of our population is,

metro.shape[0]

999999

In [21]:
# We'll make a sample of size 10,000

sample = metro.Oil_temperature[np.random.randint(low = 0, high = metro.shape[0], size = 10_000)].to_numpy()

In [23]:
# Now we have a sample os size 10,000
# Our best estimate is the sample mean,

best_estimate = sample.mean()
best_estimate

60.88413500000001

In [25]:
# Now we need a confidence interval,
# Since we are interested in the population mean, we need to use a t instead of a Z multiplier.

# Let's say that we'd like a 95% confidence interval,
# that implies, if we sampled the population a 100 times, truly randomly, creating simple random samples,
# the confidence intervals computed using the sample parameter (sample mean) will cover the true population parameter (population mean) 95 out of 100 
# times.

# confidence interval = best estimate +- margin of error
# margin of error = multiplier x standard error

# t multiplier, at 95% confidence level, for a sample of size 10,000 is,
# given our sample is succinctly large, which is true because we samples 1/100 th of the whole population, CLT becomes valid,
# which assures that the Z scores will follow a student's t distribution.

# The probability of the needed quantile covering 95% of the density of the t distribution (assumed normal) needs to be computed.
# P(-t < quantile < +t) = 0.95
# This P equals to 1 - ((1 - alpha) / 2), where alpha is the confidence.
# @ 95% confidence alpha = 0.95

P = 1 - ((1 - 0.95) / 2)
P

0.975

In [27]:
t_multiplier = t.ppf(P, df = sample.size - 1)
t_multiplier

1.9602012636213575

In [28]:
# For means standard error = sample standard deviation / square root of sample size

stderr = sample.std() / np.sqrt(sample.size)
stderr

0.06685538940637097

In [29]:
margin_of_err = t_multiplier * stderr
margin_of_err

0.13105001879426628

In [31]:
# And here's the 95% confidence interval, for population mean.

best_estimate - margin_of_err, best_estimate + margin_of_err

(60.75308498120574, 61.01518501879428)

## ___Estimating a Confidence Interval for Population Mean, Considering The Dataset as a Sample.___
----------------------

In [6]:
# That's our sample size, we do not know the population size.

metro.Oil_temperature.size

999999

In [7]:
# Our best estimate for population mean will be our sample mean.

best_est = metro.Oil_temperature.mean()
best_est

60.91453961453962

In [8]:
# To construct a 90% confidence interval, we need the margin of error.
# Since the parameter of interest here is mean, we'd refer t distribution for our multiplier.
# because our sample is fairly large with 999,999 elements.

# Since we have a large sample, we can assume that CLT applies to our sample parameters.
# To compute the quantile that covers 90% of the density of the t distribution (assumed normal),
# P = 1 - ((1 - alpha) / 2)

P = 1 - ((1 - 0.90) / 2)
P

0.95

In [9]:
# t multiplier,

t_star = t.ppf(P, df = metro.Oil_temperature.size - 1)
t_star

1.6448551507250875

In [10]:
# Standard error = sample standard deviation / square root of sample size

stderr = metro.Oil_temperature.std() / np.sqrt(metro.Oil_temperature.size)
stderr

0.006738252131068929

In [11]:
# Margin of error

moerror = t_star * stderr
moerror

0.011083448724673026

In [12]:
# 90% confidence interval

best_est - moerror, best_est + moerror

(60.903456165814944, 60.9256230632643)

## ___A Case Study Using NHANES.___
-----------------

In [14]:
nhanes = pd.read_csv("./../data/nhanes_2015_2016.csv", usecols = ["SMQ020", "BMXBMI", "RIAGENDR"])

In [15]:
nhanes = nhanes.rename({"SMQ020": "smoker", "BMXBMI": "bmi", "RIAGENDR": "gender"}, axis = 1)
nhanes

,smoker,gender,bmi
0,1,1,27.8
1,1,1,30.8
2,1,1,28.8
3,2,2,42.4
4,2,2,20.3
...,...,...,...
5730,1,2,21.5
5731,2,1,33.8
5732,1,2,31.0
5733,1,1,26.0


In [16]:
nhanes.gender.unique()

array([1, 2], dtype=int64)

In [17]:
nhanes.smoker.unique()

array([1, 2, 7, 9], dtype=int64)

In [18]:
nhanes.gender = nhanes.gender.apply(lambda g: 'M' if g == 1 else 'F')
nhanes.smoker = nhanes.smoker.apply(lambda s: False if s == 2 else True if s == 1 else np.nan)

In [19]:
nhanes.isna().sum(axis = 0)

smoker    10
gender     0
bmi       73
dtype: int64

In [20]:
nhanes.dropna(axis = 0, inplace = True)

In [21]:
nhanes

,smoker,gender,bmi
0,True,M,27.8
1,True,M,30.8
2,True,M,28.8
3,False,F,42.4
4,False,F,20.3
...,...,...,...
5730,True,F,21.5
5731,False,M,33.8
5732,True,F,31.0
5733,True,M,26.0


In [22]:
pd.crosstab(nhanes.gender, nhanes.smoker)

smoker,False,True
gender,,
F,2044,896
M,1322,1390


In [23]:
# It looks like a larger proportion of men are smokers compared to women.
# And women generally have a higher BMI compared to men.

nhanes.groupby("gender").mean()

,smoker,bmi
gender,,
F,0.304762,29.943571
M,0.512537,28.774668


In [24]:
# Additionally, smokers in bothe genders tend to have a higher BMI compared to their non-smoking counterparts.

nhanes.groupby(["gender", "smoker"]).mean()

bmi
gender smoker           
F      False   29.661057
       True    30.588058
M      False   28.592511
       True    28.947914

## ___Confidence Intervals: The Difference Between Population Proportions.___
---------------------------

In [27]:
# Consider our dataset to be a simple random sample

nhanes = pd.read_csv("./../data/nhanes_2015_2016.csv", usecols = ["SMQ020", "BMXBMI", "RIAGENDR"])
nhanes

,SMQ020,RIAGENDR,BMXBMI
0,1,1,27.8
1,1,1,30.8
2,1,1,28.8
3,2,2,42.4
4,2,2,20.3
...,...,...,...
5730,1,2,21.5
5731,2,1,33.8
5732,1,2,31.0
5733,1,1,26.0


In [30]:
# Proportion of males and females in our sample are,
# These are our best estimates for the population proportions.

prop_m = (nhanes.RIAGENDR == 1).mean()
prop_f = (nhanes.RIAGENDR == 2).mean()

prop_m, prop_f

(0.4810810810810811, 0.518918918918919)

In [32]:
# The difference between the proportions,
# That's our best estimate for the difference between the population proportions.

best_est_prop_diff = prop_f - prop_m
best_est_prop_diff

0.03783783783783784

In [33]:
# now, we need a 95% confidence interval for the difference between population proportions of males and females.

# confidence interval = best estimate +- margin of error
# margin of error = multiplier x standard error

# Since we are comparing two samples here, (the difference between the proportion of males andf females)
# We'd use a Z score of 1.96 as the multiplier, for a 95% confidence interval.

msize = (nhanes.RIAGENDR == 1).sum()
fsize = (nhanes.RIAGENDR == 2).sum()

msize, fsize

(2759, 2976)

In [34]:
# Standard error for population proportion differences is given as

# ___$SE_{prop} = \sqrt{\frac{prop (1 - prop)}{n}}$___
# ___$SE_{diff} = SE_{pop_1} + SE_{pop_2}$___

In [37]:
# Note: we are computing standard errors for population proportions.

stderr_m = np.sqrt(prop_m * (1 - prop_m) / msize)
stderr_f = np.sqrt(prop_f * (1 - prop_f) / fsize)

stderr_m, stderr_f

(0.009512245298682376, 0.00915888124615093)

In [42]:
stderr_diff = stderr_m + stderr_f
stderr_diff

0.018671126544833307

In [43]:
# Margin of erorr for the difference in population proportions,

moerr_diff = 1.96 * stderr_diff
moerr_diff

0.03659540802787328

In [44]:
# Confidence interval,

best_est_prop_diff - moerr_diff, best_est_prop_diff + moerr_diff

(0.001242429809964557, 0.07443324586571112)

## ___Confidence Intervals: The Difference Between Population Means.___
---------------------------

In [46]:
# Best estimate => difference between sample means

mean_m = nhanes.loc[nhanes.RIAGENDR == 1, "BMXBMI"].mean()
mean_f = nhanes.loc[nhanes.RIAGENDR == 2, "BMXBMI"].mean()

mean_m, mean_f

(28.778072111846942, 29.93994565217392)

In [50]:
best_est = mean_f - mean_m
best_est

1.161873540326976

In [47]:
# Sample sizes

msize = (nhanes.RIAGENDR == 1).sum()
fsize = (nhanes.RIAGENDR == 2).sum()

msize, fsize

(2759, 2976)

In [48]:
# Standard error for difference in population means,

stderr_m = nhanes.loc[nhanes.RIAGENDR == 1, "BMXBMI"].std() / np.sqrt(msize)
stderr_f = nhanes.loc[nhanes.RIAGENDR == 2, "BMXBMI"].std() / np.sqrt(fsize)

stderr_m, stderr_f

(0.11903715722332024, 0.14212522940758332)

In [49]:
# Unpooled approach,

stderr_diff = stderr_m + stderr_f
stderr_diff

0.2611623866309036

In [51]:
# Margin of error = t x standard error of difference

# For a 95% confidence interval,
# having a large sample, we'll reference CLT, assuming the t distribution of our sample means is normal,

P = 1 - ((1 - 0.95) / 2)
P

0.975

In [54]:
msize < fsize

True

In [55]:
tstar = t.ppf(P, df = msize - 1)
tstar

1.9608244975757283

In [56]:
moerr_diff = tstar * stderr_diff
moerr_diff

0.5120936055512196

In [57]:
# Confidence interval,

best_est - moerr_diff, best_est + moerr_diff

(0.6497799347757564, 1.6739671458781955)